# Credit Risk Model

# Train-Test Split
# Development - Validation and Hold-put

In [37]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

In [38]:
features=["debt_to_income_ratio","credit_score","savings_to_income_ratio"]
X=df[features]

In [39]:
X_dev, X_temp = train_test_split(X, test_size=0.4, random_state=42)
X_val, X_hold = train_test_split(X_temp, test_size = 0.5, random_state=42)

In [40]:
print(f" Development set:{len(X_dev)}rows")
print(f" Validation set:{len(X_val)}rows")
print(f" Hold-out set:{len(X_hold)}rows")

 Development set:19454rows
 Validation set:6485rows
 Hold-out set:6485rows


In [41]:
#scaling for K-Means

In [42]:
scaler= StandardScaler()

In [43]:
X_dev_scaled = scaler.fit_transform(X_dev)
X_val_scaled = scaler.transform(X_val)
X_hold_scaled = scaler.transform(X_hold)

In [44]:
#fit clustering on development set

In [45]:
Kmeans = KMeans(n_clusters=3,random_state=42)
Kmeans.fit(X_dev_scaled)

,n_clusters,3
,init,'k-means++'
,n_init,'auto'
,max_iter,300
,tol,0.0001
,verbose,0
,random_state,42
,copy_x,True
,algorithm,'lloyd'


In [46]:
X_dev_labels = Kmeans.predict(X_dev_scaled)
X_val_labels = Kmeans.predict(X_val_scaled)
X_hold_labels = Kmeans.predict(X_hold_scaled)

In [ ]:
dev_profiles = pd.DataFrame(X_dev, columns=features)
dev_profiles['cluster'] = X_dev_labels
cluster_summary = dev_profiles.groupby('cluster')[features].mean()
print(cluster_summary) 

         debt_to_income_ratio  credit_score  savings_to_income_ratio
cluster                                                             
0                    0.814711    436.574417                 5.057568
1                    0.838440    712.195431                 5.015510
2                   23.512739    581.904290                 5.218020


In [48]:
risk_mapping = {
    cluster_summary['debt_to_income_ratio'].idxmax(): 'High Risk',
    cluster_summary['debt_to_income_ratio'].idxmin(): 'Low Risk'
}

In [49]:
for c in set(range(3)) - set(risk_mapping.keys()):
    risk_mapping[c] = 'Medium Risk'

dev_profiles['risk_category'] = dev_profiles['cluster'].map(risk_mapping)

In [50]:
dev_profiles['risk_score'] = (
    (dev_profiles['debt_to_income_ratio'] / dev_profiles['debt_to_income_ratio'].max()) -
    (dev_profiles['credit_score'] / dev_profiles['credit_score'].max()) -
    (dev_profiles['savings_to_income_ratio'] / dev_profiles['savings_to_income_ratio'].max())
)

In [51]:
val_profiles = pd.DataFrame(X_val, columns=features)
val_profiles['cluster'] = X_val_labels
val_profiles['risk_category'] = val_profiles['cluster'].map(risk_mapping)

hold_profiles = pd.DataFrame(X_hold, columns=features)
hold_profiles['cluster'] = X_hold_labels
hold_profiles['risk_category'] = hold_profiles['cluster'].map(risk_mapping)

In [53]:
print(dev_profiles.sample(10))

       debt_to_income_ratio  credit_score  savings_to_income_ratio  cluster  \
24041                  1.00           693                     5.23        1   
3331                   0.00           424                     0.81        0   
7987                   0.00           404                     2.44        0   
20537                 20.42           769                     7.97        2   
6107                   0.86           764                     2.35        1   
15621                  1.97           411                     2.42        0   
2311                   0.00           410                     4.29        0   
2843                   1.20           401                     9.47        0   
16006                  0.00           686                     2.22        1   
15146                  0.00           733                     1.83        1   

      risk_category  risk_score  
24041   Medium Risk   -1.326102  
3331       Low Risk   -0.579824  
7987       Low Risk   -0.719

,debt_to_income_ratio,credit_score,savings_to_income_ratio,cluster,risk_category,risk_score
17895,0.18,467,9.12,0,Low Risk,-1.459217
361,0.00,722,7.88,1,Medium Risk,-1.637412
9217,0.00,763,7.88,1,Medium Risk,-1.685647
17691,0.00,489,5.11,0,Low Risk,-1.086294
16697,0.00,803,2.65,1,Medium Risk,-1.209706
3870,4.81,539,3.01,0,Low Risk,-0.876473
12796,0.95,770,9.73,1,Medium Risk,-1.867300
17762,1.57,310,5.43,0,Low Risk,-0.888564
6577,0.41,487,0.80,0,Low Risk,-0.647942
19698,4.88,326,2.85,0,Low Risk,-0.609032
